In [ ]:
!pip install onnx onnxruntime-gpu num2words

In [ ]:
! git clone https://github.com/huggingface/lerobot.git
! cd lerobot && pip install .

In [3]:
%pip install -q ipdb
%env PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb
%pdb on

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.1 MB/s eta 0:00:00
env: PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb
Automatic pdb calling has been turned ON


In [4]:
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig

config = SmolVLAConfig
policy = SmolVLAPolicy.from_pretrained("lerobot/smolvla_base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

Loading  HuggingFaceTB/SmolVLM2-500M-Video-Instruct weights ...


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/868 [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Reducing the number of VLM layers to 16 ...


model.safetensors:   0%|          | 0.00/907M [00:00<?, ?B/s]

In [5]:
import torch
from torch import nn
from lerobot.policies.smolvla.smolvlm_with_expert import SmolVLMWithExpertModel

from typing import Optional, List

In [22]:
class StateProjector(nn.Module):
  def __init__(self):
    super().__init__()
    self.module = policy.model.state_proj

  def forward(self, state):
    return self.module(state)

In [53]:
class ActionProjector(nn.Module):
  def __init__(self, mode="IN"):
    super().__init__()
    if mode == "IN":
      self.module = policy.model.action_in_proj
    elif mode == "OUT":
      self.module = policy.model.action_out_proj
    elif mode == "TIME_IN":
      self.module = policy.model.action_time_mlp_in
    elif mode == "TIME_OUT":
      self.module = policy.model.action_time_mlp_out

  def forward(self, state):
    return self.module(state)

In [6]:
class SmolVLMVision(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

    def forward(self, image: torch.Tensor):
        return self.model.embed_image(image)

In [7]:
class SmolVLMText(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

    def forward(self, tokens: torch.Tensor):
        return self.model.embed_language_tokens(tokens)

In [8]:
class SmolVLMForONNXExport(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

        self.config = SmolVLAConfig

        # Bake in the configuration for the static graph
        self.attention_mode = self.config.vlm_model_name
        self.num_vlm_layers = self.model.num_vlm_layers
        self.num_expert_layers = self.model.num_expert_layers
        self.fill_kv_cache = False
        self.use_cache = True

    def embed_img(self,
                  image: torch.Tensor):
          return self.model.embed_image(image)

    def embed_text(self,
                   tokens: torch.Tensor):
        return self.model.embed_language_tokens(tokens)

    def forward(
          self,
          vlm_embeds: torch.Tensor,
          expert_embeds: torch.Tensor,
          attention_mask: torch.Tensor,
          position_ids: torch.LongTensor,
          # Flattened past_key_values: one tensor for keys, one for values per layer
          *past_key_values_flat: torch.Tensor):


          inputs_embeds = [vlm_embeds, expert_embeds]

          past_key_values = {}
          if past_key_values_flat:
              # Logic to reconstruct the dictionary from the flat tuple of tensors
              # This assumes a consistent ordering: (key_layer0, val_layer0, key_layer1, val_layer1, ...)
              for i in range(0, len(past_key_values_flat), 2):
                  layer_idx = i // 2
                  past_key_values[layer_idx] = {
                      "key_states": past_key_values_flat[i],
                      "value_states": past_key_values_flat[i+1],
                  }
          else:
              past_key_values = None


          final_embeds_list, new_past_key_values = self.model.forward(
              attention_mask=attention_mask,
              position_ids=position_ids,
              past_key_values=past_key_values,
              inputs_embeds=inputs_embeds,
              use_cache=self.use_cache,
              fill_kv_cache=self.fill_kv_cache,
          )

          # --- 3. Flatten Outputs ---
          # The output must be a flat tuple of tensors
          vlm_output_embeds = final_embeds_list[0]
          expert_output_embeds = final_embeds_list[1] # expects to be None

          present_key_values_flat = []
          if new_past_key_values:
            for i in range(self.num_vlm_layers):
                if i in new_past_key_values:
                    present_key_values_flat.append(new_past_key_values[i]["key_states"])
                    present_key_values_flat.append(new_past_key_values[i]["value_states"])

          return (vlm_output_embeds, expert_output_embeds, *present_key_values_flat)

In [9]:
import torch

onnx_model = SmolVLMForONNXExport("")

PrefilL model condifuration

In [10]:
onnx_model.eval()

onnx_model.fill_kv_cache = True

onnx_model.to("cuda:0")

# output_names = ["vlm_output_embeds", "expert_output_embeds"]
output_names = ["vlm_output_embeds"]
num_layers = onnx_model.model.num_vlm_layers
for i in range(num_layers):
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size

batch_size = 1
vlm_seq_len = 256
expert_seq_len = 16
total_seq_len = vlm_seq_len + expert_seq_len

dummy_vlm_embeds = torch.randn(batch_size, vlm_seq_len, 960, device="cuda:0", dtype=torch.bfloat16)
# dummy_expert_embeds = torch.randn(batch_size, expert_seq_len, 320, device="cuda:0", dtype=torch.bfloat16) # input_embeds
dummy_expert_embeds = torch.randn(batch_size, expert_seq_len, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16) # input_embeds
# dummy_expert_embeds = torch.randn(batch_size, 320, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16) # input_embeds
dummy_attn_mask = torch.ones(batch_size, total_seq_len, total_seq_len, device="cuda:0", dtype=torch.bool)
dummy_pos_ids = torch.arange(total_seq_len, device="cuda:0").unsqueeze(0)


# dummy_inputs = (dummy_vlm_embeds, dummy_expert_embeds, dummy_attn_mask, dummy_pos_ids)
dummy_inputs = (dummy_vlm_embeds, None, dummy_attn_mask, dummy_pos_ids)

# we ant past_key_values from it, the second output



In [11]:
torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_prefill.onnx",
    input_names=["vlm_embeds", "expert_embeds", "attention_mask", "position_ids"],
    output_names=output_names,
    opset_version=17
)

/usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/smolvlm_with_expert.py:237: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_len < position_ids.shape[1]:


In [12]:
_, _, *past_key_values = onnx_model.forward(*dummy_inputs)

In [13]:
past_key_values[0].shape

torch.Size([1, 256, 5, 64])

In [14]:
import torch


onnx_model = SmolVLMForONNXExport("")

Decode model configuration and export

In [15]:

onnx_model.to("cuda:0")
onnx_model.eval()
onnx_model.fill_kv_cache = False
onnx_model.use_cache = True
current_seq_len = 1
past_seq_len = 256

batch_size = 1

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size
num_layers = onnx_model.model.num_vlm_layers
num_kv_heads = onnx_model.model.config.text_config.num_key_value_heads
head_dim = vlm_hidden_dim // onnx_model.model.config.text_config.num_attention_heads

dummy_vlm_embeds = torch.randn(batch_size, current_seq_len, 960, device="cuda:0", dtype=torch.bfloat16)
dummy_expert_embeds = torch.randn(batch_size, current_seq_len, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16)
# dummy_attn_mask = torch.ones(batch_size, current_seq_len, past_seq_len + current_seq_len, device="cuda:0", dtype=torch.bool)
dummy_attn_mask = torch.ones(batch_size, current_seq_len, current_seq_len, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(batch_size, 1, past_seq_len, past_seq_len + current_seq_len + 1, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(batch_size, 1, past_seq_len + current_seq_len, device="cuda:0", dtype=torch.bool)
# dummy_pos_ids = torch.arange(past_seq_len + current_seq_len, device="cuda:0").unsqueeze(0)
dummy_pos_ids = torch.arange(current_seq_len, device="cuda:0").unsqueeze(0)

print(past_seq_len + current_seq_len)
print(total_seq_len)

input_names = ["vlm_embeds", "expert_embeds", "attention_mask", "position_ids"]

# past_key_values_flat = []
# for _ in range(num_layers):
#     key_shape = (batch_size, num_kv_heads, past_seq_len, head_dim)
#     val_shape = (batch_size, num_kv_heads, past_seq_len, head_dim)
#     past_key_values_flat.append(torch.randn(key_shape, device="cuda:0", dtype=torch.bfloat16))
#     past_key_values_flat.append(torch.randn(val_shape, device="cuda:0", dtype=torch.bfloat16))

dummy_inputs = (
    # dummy_vlm_embeds,
    None,
    dummy_expert_embeds,
    dummy_attn_mask,
    dummy_pos_ids,
    *past_key_values
    # None
)


for i in range(num_layers):
    input_names.append(f"past_key_{i}")
    input_names.append(f"past_value_{i}")


# Outputs include the embeddings and the *updated* KV cache
# output_names = ["vlm_output_embeds", "expert_output_embeds"]
output_names = ["expert_output_embeds"]
for i in range(num_layers):
    # The output names must be consistent for the next iteration
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

# dynamic_axes = {"vlm_embeds": {}, "expert_embeds": {}} # Add other static inputs if needed
# for i in range(num_layers):
#     # The '2' axis is the sequence length dimension in the KV cache shape
#     # (batch_size, num_heads, sequence_length, head_dim)
#     dynamic_axes[f"past_key_{i}"] = {2: "past_sequence_length"}
#     dynamic_axes[f"past_value_{i}"] = {2: "past_sequence_length"}
#     dynamic_axes[f"present_key_{i}"] = {2: "total_sequence_length"}
#     dynamic_axes[f"present_value_{i}"] = {2: "total_sequence_length"}

# we need output_embeds - first output
torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_decode.onnx",
    input_names=input_names,
    output_names=output_names,
    # dynamic_axes=dynamic_axes,
    opset_version=17,
    do_constant_folding=True,
)


257
272


In [16]:
onnx_model.model.get_vlm_model().vision_model

SmolVLMVisionTransformer(
  (embeddings): SmolVLMVisionEmbeddings(
    (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
    (position_embedding): Embedding(1024, 768)
  )
  (encoder): SmolVLMEncoder(
    (layers): ModuleList(
      (0-11): 12 x SmolVLMEncoderLayer(
        (self_attn): SmolVLMVisionAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): SmolVLMVisionMLP(
          (activation_fn): PytorchGELUTanh()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (layer_norm2): LayerNorm((768,)

In [17]:
dummy_image = torch.ones(1, 3, 256, 256, device="cuda:0", dtype=torch.bfloat16) * 255
img_model = SmolVLMVision("")
img_model.to("cuda:0")
img_model.eval()

emb = img_model.forward(dummy_image)
print(emb.shape)

torch.onnx.export(
    img_model,
    dummy_image,
    "smolvlm_vision.onnx",
    input_names=["image"],
    output_names=["output_embeds"],
    opset_version=17,
    do_constant_folding=False,
)

torch.Size([1, 16, 960])


/usr/local/lib/python3.12/dist-packages/transformers/models/smolvlm/modeling_smolvlm.py:145: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for batch_idx, p_attn_mask in enumerate(patch_attention_mask):
/usr/local/lib/python3.12/dist-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask
/usr/local/lib/python3.12/dist-packages/transformers/models/smolvlm/modeling_smolvlm.py:532: TracerWarning: Converting a tensor to a Python integer might cause the 

In [18]:
text_model = SmolVLMText("").to(torch.float32).cuda()
processor = text_model.model.processor
tokens = processor(text="Hello this is a test text prompt", return_tensors="pt")
print(f"Text tokens = {tokens}")

embs = text_model.forward(tokens["input_ids"].cuda())
print(f"Shpae of text embeddings = {emb.shape}")

torch.onnx.export(
    text_model,
    tokens["input_ids"].cuda(),
    "smolvlm_text.onnx",
    input_names=["tokens"],
    output_names=["output_embeds"],
    dynamic_axes={"tokens": {0: "batch", 1: "sequence_length"}, "output_embeds": {0: "batch", 1: "sequence_length"}},
    opset_version=17,
)

Text tokens = {'input_ids': tensor([[19556,   451,   314,   253,  1028,  1694,  6011]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
Shpae of text embeddings = torch.Size([1, 16, 960])


Check if onnx outputs and torch outputs are equal

In [19]:
import onnxruntime as ort
import numpy as np

# providers = ["CUDAExecutionProvider"] if torch.cuda.is_available() else ["CPUExecutionProvider"]
providers = ["CPUExecutionProvider"]

text_session = ort.InferenceSession("smolvlm_text.onnx", providers=providers)

tokens = processor(text="Hello this is a test text prompt", return_tensors="np")

input_name = text_session.get_inputs()[0].name

onnx_outputs = text_session.run(None, {input_name: tokens["input_ids"]})

# The output is a list, get the first element which should be the embeddings
onnx_embs = onnx_outputs[0]

assert((embs.cpu() == onnx_embs).all())

In [36]:
# policy.config.n_obs_steps
policy.config.input_features['observation.state'].shape

(6,)

In [63]:
state_projector = StateProjector().cuda()
input = torch.randn(1, policy.config.input_features['observation.state'].shape[0], 32, device="cuda:0") # shape seems correct, but need to check it further

state_projector.eval()

print(state_projector.forward(input).shape)

state_projector.forward(input)

torch.onnx.export(
    state_projector,
    input,
    "state_projector.onnx",
    input_names=["state"],
    opset_version=17)

torch.Size([1, 6, 960])


In [47]:
action_in_projector = ActionProjector(mode="IN").cuda()
input = torch.randn(1, policy.config.input_features['observation.state'].shape[0], 32, device="cuda:0") # shape seems correct, but need to check it further

action_in_projector.eval()

action_in_projector.forward(input)

torch.onnx.export(
    action_in_projector,
    input,
    "action_in_projector.onnx",
    input_names=["state"],
    opset_version=17)

In [50]:
action_out_projector = ActionProjector(mode="OUT").cuda()
# input = torch.randn(1, policy.config.input_features['observation.state'].shape[0], policy.config.n_obs_steps)
input = torch.randn(1, policy.config.input_features['observation.state'].shape[0], 720, device="cuda:0") # TODO check for the actual shape

action_out_projector.eval()

print(action_out_projector.forward(input).shape)

torch.onnx.export(
    action_out_projector,
    input,
    "action_out_projector.onnx",
    input_names=["action"],
    opset_version=17)

torch.Size([1, 6, 32])


In [59]:
time_out_projector = ActionProjector(mode="TIME_OUT").cuda()
input = torch.randn(1, 1, 720, device="cuda:0") # TODO check for the actual shape

time_out_projector.eval()

print(time_out_projector.forward(input).shape)

torch.onnx.export(
    time_out_projector,
    input,
    "time_out_projector.onnx",
    input_names=["time"],
    opset_version=17)

torch.Size([1, 1, 720])


In [60]:
time_in_projector = ActionProjector(mode="TIME_IN").cuda()
input = torch.randn(1, 1, 1440, device="cuda:0") # TODO check for the actual shape

time_in_projector.eval()

print(time_in_projector.forward(input).shape)

torch.onnx.export(
    time_in_projector,
    input,
    "time_in_projector.onnx",
    input_names=["time"],
    opset_version=17)

torch.Size([1, 1, 720])


In [20]:
import os, onnx

out_name = None
src_name = "smolvlm_expert_prefill.onnx"
out_dir = "splited"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )

In [21]:
import os, onnx

out_name = None
src_name = "smolvlm_expert_decode.onnx"
out_dir = "splited_decoder"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )